In [2]:
import torch

# Preprocessing

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,LabelEncoder

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [5]:
df.shape

(569, 33)

In [6]:
df.drop(columns=['id','Unnamed: 32'],inplace=True)

In [7]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [8]:
xtrain,xtest,ytrain,ytest=train_test_split(df.iloc[:,1:],df.iloc[:,0],test_size=0.2)

In [9]:
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)

In [10]:
ytrain[:10]

560    B
97     B
42     M
195    B
75     M
454    B
41     M
434    B
4      M
537    B
Name: diagnosis, dtype: object

In [11]:
encoder=LabelEncoder()
ytrain=encoder.fit_transform(ytrain)
ytest=encoder.transform(ytest)

In [12]:
ytrain[:10]

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [13]:
xtraintensor=torch.from_numpy(xtrain)
xtesttensor=torch.from_numpy(xtest)
ytraintensor=torch.from_numpy(ytrain)
ytesttensor=torch.from_numpy(ytest)

In [14]:
xtraintensor.shape

torch.Size([455, 30])

# Neural Netork part

In [18]:
class MySimpleNN():
    def __init__(self,x):
        self.weights=torch.rand(x.shape[1],1,dtype=torch.float64,requires_grad=True)
        self.bias=torch.zeros(1,dtype=torch.float64,requires_grad=True)

    def forward(self,x):
        z=torch.matmul(x,self.weights)+self.bias
        return torch.sigmoid(z)

    def lossfunction(self,ypred,ytraintensor):
        eps=1e-7
        ypred=torch.clamp(ypred,eps,1-eps)
        loss=-(ytraintensor*torch.log(ypred)+(1-ytraintensor)*torch.log(1-ypred)).mean()
        return loss

In [19]:
lr=0.1
epochs=25

In [20]:
model=MySimpleNN(xtraintensor)

In [21]:
for epoch in range(epochs):
    ypred=model.forward(xtraintensor)
    loss=model.lossfunction(ypred,ytraintensor)
    loss.backward()
    with torch.no_grad():
        model.weights-=lr*model.weights.grad
        model.bias-=lr*model.bias.grad
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    print(f'epoch: {epoch+1}, loss: {loss.item()}')   

epoch: 1, loss: 3.539438314811865
epoch: 2, loss: 3.4151970842878945
epoch: 3, loss: 3.2901517859577156
epoch: 4, loss: 3.1639408705042915
epoch: 5, loss: 3.0303792613570906
epoch: 6, loss: 2.8928400484532264
epoch: 7, loss: 2.7494423574197535
epoch: 8, loss: 2.605058923355694
epoch: 9, loss: 2.452797703337886
epoch: 10, loss: 2.294810151314784
epoch: 11, loss: 2.1359323507071797
epoch: 12, loss: 1.9716294789758582
epoch: 13, loss: 1.8105015624434881
epoch: 14, loss: 1.6540753770657948
epoch: 15, loss: 1.5073846002898263
epoch: 16, loss: 1.3690883356793857
epoch: 17, loss: 1.2448670242524429
epoch: 18, loss: 1.1363725456923035
epoch: 19, loss: 1.044876364570219
epoch: 20, loss: 0.970835552666486
epoch: 21, loss: 0.9134078390483171
epoch: 22, loss: 0.8703058486754057
epoch: 23, loss: 0.8384006027621801
epoch: 24, loss: 0.8146661606058264
epoch: 25, loss: 0.7966849217302144


In [22]:
model.weights

tensor([[ 0.5050],
        [ 0.4214],
        [-0.3819],
        [ 0.0807],
        [ 0.2389],
        [ 0.0011],
        [ 0.1146],
        [-0.2942],
        [-0.1918],
        [-0.1251],
        [-0.2981],
        [ 0.5774],
        [-0.0129],
        [-0.1059],
        [ 0.5455],
        [ 0.1961],
        [ 0.0138],
        [-0.3162],
        [ 0.1174],
        [-0.1178],
        [-0.1161],
        [-0.1974],
        [-0.1606],
        [ 0.3119],
        [-0.0968],
        [-0.0392],
        [ 0.3197],
        [ 0.1540],
        [ 0.2987],
        [ 0.0678]], dtype=torch.float64, requires_grad=True)

In [23]:
model.bias

tensor([-0.1395], dtype=torch.float64, requires_grad=True)

In [24]:
with torch.no_grad():
    ypred=model.forward(xtesttensor)
    ypred=(ypred>0.8).float()
    acc=(ypred==ytesttensor).float().mean()
    print(f'acc :{acc}')

acc :0.604032039642334
